In [14]:
# Install dependencies if missing (uncomment as needed)
# !pip install geopandas folium pandas fiona shapely pyproj
import geopandas as gpd
import pandas as pd
from pathlib import Path
import folium
from folium.plugins import MarkerCluster, FastMarkerCluster
import math

BASE = Path('NY Crimes')  # base data directory
PSA_FILE = BASE / 'NYCHA PSA (Police Service Areas).geojson'
SECTORS_FILE = BASE / 'NYPD Sectors.geojson'
PRECINCTS_FILE = BASE / 'Police Precincts.geojson'
COMPLAINTS_2019_CSV = BASE / 'NYPD_Complaint_Data_Historic_2019.csv'  # fallback CSV
GDB_PATH = BASE / 'ny_data.gdb'  # optional feature class source
FEATURECLASS_NAME = 'NYPD_Complaint_Data_Historic_2019_4326'  # expected point layer name

# Central coordinates (NYC approximate)
NYC_LAT, NYC_LON = 40.7128, -74.0060

# Performance knobs
MAX_POINTS = 10000  # limit point markers to avoid huge HTML

print('Data directory exists:', BASE.exists())

Data directory exists: True


In [15]:
# Helper utilities
def simplify_geometries(gdf: gpd.GeoDataFrame, tolerance: float = 0.001) -> gpd.GeoDataFrame:
    """Return copy with simplified geometries (project to EPSG:3857 for metric simplification)."""
    if gdf.crs is None:
        return gdf
    try:
        projected = gdf.to_crs(3857)
        simplified_geom = projected.geometry.simplify(tolerance, preserve_topology=True)
        out = projected.copy()
        out.geometry = simplified_geom
        return out.to_crs(4326)
    except Exception as e:
        print('Simplification failed:', e)
        return gdf

def safe_read_gdb(gdb_path: Path, layer: str):
    try:
        return gpd.read_file(gdb_path, layer=layer)
    except Exception as e:
        print(f'GDB read failed ({layer}):', e)
        return None

def detect_coordinate_columns(df: pd.DataFrame):
    cands_lat = [c for c in df.columns if c.lower() in ('latitude','lat','y')]
    cands_lon = [c for c in df.columns if c.lower() in ('longitude','lon','lng','x')]
    return (cands_lat[0] if cands_lat else None, cands_lon[0] if cands_lon else None)

def try_build_points(df: pd.DataFrame, lat_col: str, lon_col: str):
    subset = df[[lat_col, lon_col]].dropna()
    # basic validity filter for NYC bounding box
    subset = subset[(subset[lat_col].between(40.3, 41.0)) & (subset[lon_col].between(-74.5, -73.5))]
    gdf = gpd.GeoDataFrame(subset, geometry=gpd.points_from_xy(subset[lon_col], subset[lat_col]), crs='EPSG:4326')
    return gdf

def sanitize_properties(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Cast non-JSON-serializable types (e.g., pandas Timestamps) to strings.
    Handles timezone-aware datetimes by converting to UTC then dropping tz.
    """
    out = gdf.copy()
    for col in out.columns:
        if col == 'geometry':
            continue
        ser = out[col]
        # Datetime-like columns
        if pd.api.types.is_datetime64_any_dtype(ser) or pd.api.types.is_timedelta64_dtype(ser):
            try:
                # If timezone-aware, convert to UTC then naive
                if hasattr(ser.dtype, 'tz') and ser.dtype.tz is not None:
                    ser = ser.dt.tz_convert('UTC').dt.tz_localize(None)
                else:
                    # ensure ns precision
                    ser = ser.astype('datetime64[ns]')
                out[col] = ser.dt.strftime('%Y-%m-%d %H:%M:%S')
            except Exception as e:
                # fallback to string
                out[col] = ser.astype(str)
        else:
            # Convert pandas Timestamp objects inside object dtype
            if ser.dtype == 'object':
                def _convert(v):
                    try:
                        if hasattr(v, 'isoformat'):
                            # If tz-aware Timestamp
                            if isinstance(v, pd.Timestamp) and v.tz is not None:
                                return v.tz_convert('UTC').tz_localize(None).isoformat()
                            return v.isoformat()
                        if isinstance(v, pd.Timestamp):
                            return v.isoformat()
                        return v
                    except Exception:
                        return str(v)
                out[col] = ser.apply(_convert)
    return out

print('Helper functions loaded.')

Helper functions loaded.


In [16]:
# Load boundary GeoJSON layers
psa = gpd.read_file(PSA_FILE)
sectors = gpd.read_file(SECTORS_FILE)
precincts = gpd.read_file(PRECINCTS_FILE)

# Normalize CRS to WGS84 (Leaflet expects EPSG:4326)
for name, gdf in [('PSA', psa), ('Sectors', sectors), ('Precincts', precincts)]:
    if gdf.crs and gdf.crs.to_epsg() != 4326:
        print(f'Reprojecting {name} from {gdf.crs} to EPSG:4326')
        gdf.to_crs(4326, inplace=True)

print('Layer counts:', len(psa), len(sectors), len(precincts))
# Increase simplification tolerance to shrink HTML size (meters via 3857 projection inside helper)
psa_simplified = simplify_geometries(psa, 50)
sectors_simplified = simplify_geometries(sectors, 25)
precincts_simplified = simplify_geometries(precincts, 100)
print('Simplification complete.')

# Sanitize properties to avoid Folium JSON serialization issues
psa_simplified = sanitize_properties(psa_simplified)
sectors_simplified = sanitize_properties(sectors_simplified)
precincts_simplified = sanitize_properties(precincts_simplified)

# Prune non-essential columns to reduce HTML payload

def keep_minimal_columns(gdf, keys):
    cols = ['geometry']
    for k in keys:
        cols.extend([c for c in gdf.columns if c != 'geometry' and k in c.lower()])
    # de-duplicate, preserve order
    cols = list(dict.fromkeys(cols))
    # Always keep at least geometry
    cols = [c for c in cols if c in gdf.columns]
    return gdf[cols]

psa_simplified = keep_minimal_columns(psa_simplified, ['psa', 'name', 'id', 'label'])
sectors_simplified = keep_minimal_columns(sectors_simplified, ['sector', 'sect', 'name', 'id', 'label'])
precincts_simplified = keep_minimal_columns(precincts_simplified, ['precinct', 'prec', 'name', 'id', 'label'])

Layer counts: 16 302 77
Simplification complete.
Simplification complete.


In [17]:
# Load complaint points: prefer GDB feature class, fallback to CSV
complaints_gdf = safe_read_gdb(GDB_PATH, FEATURECLASS_NAME)
if complaints_gdf is not None:
    print('Loaded complaints from GDB feature class:', len(complaints_gdf))
    if complaints_gdf.crs and complaints_gdf.crs.to_epsg() != 4326:
        complaints_gdf = complaints_gdf.to_crs(4326)
else:
    print('Falling back to CSV for complaints.')
    df_2019 = pd.read_csv(COMPLAINTS_2019_CSV)
    lat_col, lon_col = detect_coordinate_columns(df_2019)
    if lat_col and lon_col:
        complaints_gdf = try_build_points(df_2019, lat_col, lon_col)
        print('Built point GeoDataFrame from CSV:', len(complaints_gdf))
    else:
        complaints_gdf = gpd.GeoDataFrame(columns=['geometry'], geometry=[])
        print('No coordinate columns detected; complaints layer empty.')

# Optionally sample to improve rendering performance (automatic)
if len(complaints_gdf) > MAX_POINTS:
    complaints_gdf = complaints_gdf.sample(MAX_POINTS, random_state=42)
    print(f'Sampled complaints to {len(complaints_gdf)} points for performance.')

print('Complaints CRS:', complaints_gdf.crs)

# Sanitize complaint properties too (for choropleth & potential popups)
complaints_gdf = sanitize_properties(complaints_gdf)

# Keep only geometry to reduce payload size
if 'geometry' in complaints_gdf.columns:
    complaints_gdf = complaints_gdf[['geometry']].dropna()

Loaded complaints from GDB feature class: 450976
Sampled complaints to 10000 points for performance.
Complaints CRS: EPSG:4326
Sampled complaints to 10000 points for performance.
Complaints CRS: EPSG:4326


In [18]:
# Aggregate complaints per precinct for choropleth (attempt using common precinct columns)
precinct_cols_candidates = [c for c in complaints_gdf.columns if 'precinct' in c.lower()]
choropleth_df = None
if precinct_cols_candidates:
    precinct_col = precinct_cols_candidates[0]
    # Normalize precinct ID to string for join
    counts = complaints_gdf.groupby(precinct_col).size().reset_index(name='complaint_count')
    # Attempt to find matching precinct key in precincts layer
    precinct_key_candidates = [c for c in precincts_simplified.columns if 'precinct' in c.lower() or 'prec' in c.lower()]
    if precinct_key_candidates:
        precinct_key = precinct_key_candidates[0]
        # Cast types for join
        counts[precinct_col] = counts[precinct_col].astype(str)
        precincts_simplified[precinct_key] = precincts_simplified[precinct_key].astype(str)
        choropleth_df = precincts_simplified.merge(counts, left_on=precinct_key, right_on=precinct_col, how='left')
        choropleth_df['complaint_count'].fillna(0, inplace=True)
        # Keep minimal columns for rendering
        choropleth_df = choropleth_df[['geometry', precinct_key, 'complaint_count']]
        print('Choropleth join complete. Rows:', len(choropleth_df))
    else:
        print('Could not find precinct key in precincts layer; skipping choropleth.')
else:
    print('No precinct column detected in complaints; choropleth skipped.')

No precinct column detected in complaints; choropleth skipped.


In [19]:
# Build interactive Folium map
m = folium.Map(location=[NYC_LAT, NYC_LON], zoom_start=11, tiles='cartodbpositron')

def style_func_psa(_):
    return {'color': '#1f77b4', 'weight': 1, 'fillOpacity': 0.05}
def style_func_sectors(_):
    return {'color': '#ff7f0e', 'weight': 1, 'fillOpacity': 0.05, 'dashArray': '4 4'}
def style_func_precincts(_):
    return {'color': '#2ca02c', 'weight': 1.2, 'fillOpacity': 0.04}

folium.GeoJson(psa_simplified, name='PSA', style_function=style_func_psa,
               tooltip=folium.GeoJsonTooltip(fields=[c for c in psa_simplified.columns if c.lower().startswith('psa')][:1] or None)).add_to(m)
folium.GeoJson(sectors_simplified, name='Sectors', style_function=style_func_sectors).add_to(m)
folium.GeoJson(precincts_simplified, name='Precincts', style_function=style_func_precincts).add_to(m)

# Add complaint points using FastMarkerCluster for performance
if not complaints_gdf.empty:
    coords = [[round(pt.y, 5), round(pt.x, 5)] for pt in complaints_gdf.geometry if pt and not pt.is_empty]
    if coords:
        FastMarkerCluster(coords, name='Complaint Points').add_to(m)
    else:
        print('No valid point geometries found for markers.')
else:
    print('Complaint points layer empty; skipping markers.')

# Choropleth layer if available
if choropleth_df is not None:
    precinct_key_candidates = [c for c in choropleth_df.columns if 'precinct' in c.lower() or 'prec' in c.lower()]
    if precinct_key_candidates:
        precinct_key = precinct_key_candidates[0]
        folium.Choropleth(
            geo_data=choropleth_df.__geo_interface__,
            data=choropleth_df[[precinct_key, 'complaint_count']],
            columns=[precinct_key, 'complaint_count'],
            key_on=f'feature.properties.{precinct_key}',
            fill_color='YlOrRd',
            fill_opacity=0.6,
            line_opacity=0.2,
            nan_fill_color='white',
            legend_name='Complaint Count',
            name='Complaints Choropleth'
        ).add_to(m)
    else:
        print('Precinct key missing for choropleth rendering.')

folium.LayerControl(collapsed=False).add_to(m)

## Performance & Next Steps
- If rendering is slow: increase simplification tolerance or sample points.
- Add temporal filters by extracting date columns and using widgets (e.g., ipywidgets).
- Export map to HTML: `m.save('nyc_crime_map.html')`.
- Enhance popups with offense description, time of day bins, or victim demographics.
- Integrate advanced visualization (kepler.gl, deck.gl via pydeck) for 3D density surfaces.

In [20]:
m.save('NYC_optimized_crime_map.html') 